# Convolutional Network

This notebook shows how to apply convolutional networks to image processing problems

### Import all the needed modules

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
import tensorflow as tf
import numpy as np

### Define the path of the data source for convenience

Source of the data is the `train.csv` of https://www.kaggle.com/c/digit-recognizer/data

The data with the first 25000 rows is also provided with the code in the `data` directory

In [ ]:
FILE_PATH = '../data/digits.csv'

### Load the dataset and view the first few rows

In [ ]:
df = pd.read_csv(FILE_PATH)
df.head()

### Convert the labels to one-hot encoding

This is needed when working with a multi-label problem since the network will predict the probability for each label

In [ ]:
y = df['label'].values
y[:5]

In [ ]:
y_encoder = OneHotEncoder(sparse=False)
y_encoded = y_encoder.fit_transform(y.reshape(-1, 1))
y_encoded[:5]

### Remove the labels in the set of our input features

The data is reshaped with a fourth dimension. This is because the fourth dimension represents the color bands. For example, a normal image has 3 bands (RGB) and this monochromatic data set only has one

In [ ]:
X = df.drop('label', axis=1).values
X = X.reshape(-1, 28, 28, 1)

### Verify the images

Show the data to make sure converted the csv rows to their proper images.

In [ ]:
for i in range(10):
    plt.imshow(X[i].reshape(28, 28))
    plt.show()
    print('Label:', y[i])

### Create a Keras model for training

This architecture is similar to a smaller VGG

In [ ]:
input_ = tf.keras.Input((28, 28, 1))
conv1 = tf.keras.layers.Conv2D(8, (3, 3), activation='relu')(input_)
conv2 = tf.keras.layers.Conv2D(8, (3, 3), activation='relu')(conv1)
mp1 = tf.keras.layers.MaxPool2D((2,2))(conv2)
conv3 = tf.keras.layers.Conv2D(8, (3, 3), activation='relu')(mp1)
conv4 = tf.keras.layers.Conv2D(8, (3, 3), activation='relu')(conv3)
conv5 = tf.keras.layers.Conv2D(8, (3, 3), activation='relu')(conv4)
mp2 = tf.keras.layers.MaxPool2D((2,2))(conv5)
fl = tf.keras.layers.Flatten()(mp2)
dense1 = tf.keras.layers.Dense(8, activation='relu')(fl)
output = tf.keras.layers.Dense(10, activation='softmax')(dense1)

model = tf.keras.Model(inputs=input_, outputs=output)
model.summary()

In [ ]:
model.compile('adam', 'categorical_crossentropy')

In [ ]:
tf.keras.utils.plot_model(model)

### Fit the model using the training data

It's better to use the callbacks used in the previous notebook to better training results

In [ ]:
hst = model.fit(X, y_encoded, batch_size=32, epochs=10, validation_split=0.2)

### Try out some predictions

It's up to you to try the model performance on a separate training set. The prediction is only done to validate the training operation

In [ ]:
predictions = model.predict(X)

In [ ]:
for i in range(10):
    plt.imshow(X[i].reshape(28, 28))
    plt.show()
    print('Prediction:', predictions[i])

### Convert the probabilities to labels

Get the index of the largest probability per row

In [ ]:
for i in range(10):
    plt.imshow(X[i].reshape(28, 28))
    plt.show()
    print('Prediction:', np.argmax(predictions[i]))